# Inspect the properties on Wikidata

In this notebook we will inspect the different properties of Wikidata and show which ones are interesting to study in a statistical analysis.
The most essential thing to focus on is which properties are present in the maximum number of instances. It is also important that the instances share most of the properties.

To do this, we set up a query that retrieves all the properties but first we exclude the properties including an identifier (because there is no interest to study and this reduces the number of responses). We also exclude properties with too few instances (relative to the number of instances in the query). It drastically decreases the execution time of the request.

All the queries are first written on the SQLite database to keep them in memory. (to create it see. https://github.com/Semantic-Data-for-Humanities/Economists_Jurists/blob/development/Notebooks/Merge/Database_SQlite.ipynb).

In [1]:
# Import libraries usefull

from SPARQLWrapper import SPARQLWrapper, SPARQLWrapper2, JSON, TURTLE, XML, RDFXML
import pprint
import csv
# from bs4 import BeautifulSoup

from collections import Counter
from operator import itemgetter
import pandas as pd

import sqlite3 as sql
import time

from importlib import reload
from shutil import copyfile

In [2]:
import sparql_functions as spqf # It's made-home fonctions created by Francesco Beretta
# so they must to be in the same folder as this file.

# Execution of the query

In [92]:
### It's define the database ligne to use
pk_query = 54

# Connexion to the database
original_db = 'data/sparql_queries.db'
conn = sql.connect(original_db)

c = conn.cursor()

### It runs the query on the SQLite database to get the row values 
c.execute('SELECT * FROM query WHERE pk_query = ?', [pk_query]) ### a list around argument is needed for a string longer then one
#c.execute('SELECT * FROM query WHERE pk_query = 10')

rc = c.fetchone()

# close connexion
conn.close()


In [93]:
print(rc[2] +  "\n-----\n" + rc[4] +  "\n-----\n" +   rc[7]+  "\n\n\n------------------\n" +  rc[5] + "\n\n\n------------------\n")

SELECT: This query gets all the properties of magistrate.
I only exclude the properties with less of 50 effectives and all properties with IDs
-----
https://query.wikidata.org/sparql
-----
2021-05-26 15:45:14


------------------
SELECT  ?prop ?propLabel ?direction ?eff
WHERE{ { 
        SELECT  ?prop ?propLabel ?direction ?eff
WHERE{
    {
  SELECT   ?prop ?direction  (COUNT(*) as ?eff) # ?pLabel
 WHERE
  { 
  hint:Query hint:optimizer "None" .
      { ?person  wdt:P106 wd:Q16009129. # magistrate | result:  77 people
       ?person ?p ?o.
       ?prop wikibase:directClaim ?p .
       BIND("out" AS ?direction)
      }
  UNION
       {
       ?person  wdt:P106  wd:Q16009129.
       ?s ?p ?person.
       ?prop wikibase:directClaim ?p .
       BIND("in" AS ?direction)
      }
  }

GROUP BY ?prop ?direction # ?pLabel
               }
SERVICE wikibase:label {bd:serviceParam wikibase:language "en" }
}
    }
FILTER (!regex(str(?propLabel), "ID"))
FILTER((?eff) > 10)
}
ORDER BY DESC(?eff)



-

In [94]:
### Execute the SPARQL query wrapped in the function in the library _sparql_functions.py_
# The first setting correspond to SPARQL Endpoint, the seconde to the query
q = spqf.get_json_sparql_result(rc[4],rc[5])

<class 'dict'>


In [95]:
### This fonction retrieves and shapes time

# definition
def timestamp_formatted_for_file_name():
    is_now = time.strftime('%Y%m%d_%H%M%S')
    return is_now

# execution
timestamp_formatted_for_file_name()

'20210531_150919'

In [96]:
# Define the file addresses, the existing one and new one
original_db = 'data/sparql_queries.db'

timestamped_db_copy = 'data/sparql_queries_' + timestamp_formatted_for_file_name() + '.sqlite'

In [97]:
## Documentation:
# https://docs.python.org/3/library/shutil.htmlcopied_db = copyfile(original_db, timestamped_db_copy)

copied_db = copyfile(original_db, timestamped_db_copy)
copied_db

'data/sparql_queries_20210531_150920.sqlite'

In [98]:
### store the answer of the SPARQL endpoint in the 'result'

conn = sql.connect(original_db)
c = conn.cursor()
values = (pk_query, str(q),timestamp_formatted_for_file_name())

# https://www.techonthenet.com/sqlite/functions/now.php
c.execute("INSERT INTO result (fk_query, result, timestmp) VALUES (?,?,?)", values)
# commit the insertion and close the database
# !! REQUIRE to be enabled to commit to the SQLite database !!
conn.commit()
conn.close()

# Inspect the queries

In [104]:
### Inspect the resultat after an insert

# Choose the row of the database to get
pk_result = ('15')

# connexion to the database
original_db = 'data/sparql_queries.db'
conn = sql.connect(original_db)

### execute the query on the SQLite database to retrieve the values of the row
c = conn.cursor()
c.execute('SELECT * FROM result WHERE pk_result = ?', (pk_result,))
result_q = c.fetchone()

# close connexion
conn.close()
# result_q[3]

In [105]:
### Transform string to dict
## Doc.:
# https://stackoverflow.com/questions/988228/convert-a-string-representation-of-a-dictionary-to-a-dictionary
import ast
d = ast.literal_eval(result_q[3])
type(d)

dict

In [108]:
##### Transform the result into a list with a fonction of the library #####

#### Result of the query
r_query = [l for l in spqf.sparql_result_to_list(d)]
print(len(r_query))
r_query

74


[['http://www.wikidata.org/entity/P106', 'occupation', 'out', '55'],
 ['http://www.wikidata.org/entity/P39', 'position held', 'out', '55'],
 ['http://www.wikidata.org/entity/P69', 'educated at', 'out', '47'],
 ['http://www.wikidata.org/entity/P1412',
  'languages spoken, written or signed',
  'out',
  '33'],
 ['http://www.wikidata.org/entity/P735', 'given name', 'out', '32'],
 ['http://www.wikidata.org/entity/P166', 'award received', 'out', '30'],
 ['http://www.wikidata.org/entity/P569', 'date of birth', 'out', '29'],
 ['http://www.wikidata.org/entity/P31', 'instance of', 'out', '29'],
 ['http://www.wikidata.org/entity/P21', 'sex or gender', 'out', '29'],
 ['http://www.wikidata.org/entity/P27', 'country of citizenship', 'out', '28'],
 ['http://www.wikidata.org/entity/P734', 'family name', 'out', '24'],
 ['http://www.wikidata.org/entity/P19', 'place of birth', 'out', '24'],
 ['http://www.wikidata.org/entity/P101', 'field of work', 'out', '19'],
 ['http://www.wikidata.org/entity/P213', '

## Economist

In this query, we exclude the properties with less of 5000 in number.
In total, There is 29322 instances

``
 [['http://www.wikidata.org/entity/P50', 'author', 'in', '73811'],
 ['http://www.wikidata.org/entity/P106', 'occupation', 'out', '62139'],
 ['http://www.wikidata.org/entity/P31', 'instance of', 'out', '35764'],
 ['http://www.wikidata.org/entity/P21', 'sex or gender', 'out', '35027'],
 ['http://www.wikidata.org/entity/P735', 'given name', 'out', '31056'],
 ['http://www.wikidata.org/entity/P569', 'date of birth', 'out', '30315'],
 ['http://www.wikidata.org/entity/P69', 'educated at', 'out', '28157'],
 ['http://www.wikidata.org/entity/P27',
  'country of citizenship',
  'out',
  '26878'],
 ['http://www.wikidata.org/entity/P108', 'employer', 'out', '20239'],
 ['http://www.wikidata.org/entity/P19', 'place of birth', 'out', '19449'],
 ['http://www.wikidata.org/entity/P213', 'ISNI', 'out', '18092'],
 ['http://www.wikidata.org/entity/P1412',
  'languages spoken, written or signed',
  'out',
  '17322'],
 ['http://www.wikidata.org/entity/P734', 'family name', 'out', '16033'],
 ['http://www.wikidata.org/entity/P166', 'award received', 'out', '14632'],
 ['http://www.wikidata.org/entity/P184', 'doctoral advisor', 'in', '14254'],
 ['http://www.wikidata.org/entity/P185', 'doctoral student', 'out', '14020'],
 ['http://www.wikidata.org/entity/P570', 'date of death', 'out', '11346'],
 ['http://www.wikidata.org/entity/P39', 'position held', 'out', '10814'],
 ['http://www.wikidata.org/entity/P18', 'image', 'out', '8888'],
 ['http://www.wikidata.org/entity/P102',
  'member of political party',
  'out',
  '6914'],
 ['http://www.wikidata.org/entity/P20', 'place of death', 'out', '6532'],
 ['http://www.wikidata.org/entity/P1559',
  'name in native language',
  'out',
  '6214'],
 ['http://www.wikidata.org/entity/P463', 'member of', 'out', '5497'],
 ['http://www.wikidata.org/entity/P1343',
  'described by source',
  'out',
  '5068'],
 ['http://www.wikidata.org/entity/P373', 'Commons category', 'out', '5038']]
 ``

## Jurist

In this query, we exclude the properties with less of 5000 in number.
In total, There is 24886 instances

``
[['http://www.wikidata.org/entity/P106', 'occupation', 'out', '66309'],
 ['http://www.wikidata.org/entity/P31', 'instance of', 'out', '32133'],
 ['http://www.wikidata.org/entity/P21', 'sex or gender', 'out', '31910'],
 ['http://www.wikidata.org/entity/P569', 'date of birth', 'out', '30681'],
 ['http://www.wikidata.org/entity/P735', 'given name', 'out', '30082'],
 ['http://www.wikidata.org/entity/P27',
  'country of citizenship',
  'out',
  '23782'],
 ['http://www.wikidata.org/entity/P570', 'date of death', 'out', '22391'],
 ['http://www.wikidata.org/entity/P19', 'place of birth', 'out', '22305'],
 ['http://www.wikidata.org/entity/P1412',
  'languages spoken, written or signed',
  'out',
  '17719'],
 ['http://www.wikidata.org/entity/P213', 'ISNI', 'out', '15196'],
 ['http://www.wikidata.org/entity/P20', 'place of death', 'out', '13572'],
 ['http://www.wikidata.org/entity/P39', 'position held', 'out', '13214'],
 ['http://www.wikidata.org/entity/P50', 'author', 'in', '12467'],
 ['http://www.wikidata.org/entity/P734', 'family name', 'out', '12143'],
 ['http://www.wikidata.org/entity/P69', 'educated at', 'out', '11893'],
 ['http://www.wikidata.org/entity/P166', 'award received', 'out', '9286'],
 ['http://www.wikidata.org/entity/P108', 'employer', 'out', '9180'],
 ['http://www.wikidata.org/entity/P1343',
  'described by source',
  'out',
  '7096'],
 ['http://www.wikidata.org/entity/P18', 'image', 'out', '7015'],
 ['http://www.wikidata.org/entity/P102',
  'member of political party',
  'out',
  '5276'],
 ['http://www.wikidata.org/entity/P463', 'member of', 'out', '5026']]
``

## Lawyers

In this query, we exclude the properties with less of 5000 in number.
In total, There is 68033 instances.

``
[['http://www.wikidata.org/entity/P106', 'occupation', 'out', '164319'],
 ['http://www.wikidata.org/entity/P31', 'instance of', 'out', '76518'],
 ['http://www.wikidata.org/entity/P21', 'sex or gender', 'out', '75720'],
 ['http://www.wikidata.org/entity/P569', 'date of birth', 'out', '71859'],
 ['http://www.wikidata.org/entity/P735', 'given name', 'out', '70724'],
 ['http://www.wikidata.org/entity/P27',
  'country of citizenship',
  'out',
  '67490'],
 ['http://www.wikidata.org/entity/P69', 'educated at', 'out', '65950'],
 ['http://www.wikidata.org/entity/P39', 'position held', 'out', '65654'],
 ['http://www.wikidata.org/entity/P19', 'place of birth', 'out', '54125'],
 ['http://www.wikidata.org/entity/P570', 'date of death', 'out', '47410'],
 ['http://www.wikidata.org/entity/P1412',
  'languages spoken, written or signed',
  'out',
  '41384'],
 ['http://www.wikidata.org/entity/P734', 'family name', 'out', '37816'],
 ['http://www.wikidata.org/entity/P102',
  'member of political party',
  'out',
  '33891'],
 ['http://www.wikidata.org/entity/P18', 'image', 'out', '29423'],
 ['http://www.wikidata.org/entity/P20', 'place of death', 'out', '28912'],
 ['http://www.wikidata.org/entity/P213', 'ISNI', 'out', '20163'],
 ['http://www.wikidata.org/entity/P1559',
  'name in native language',
  'out',
  '18462'],
 ['http://www.wikidata.org/entity/P937', 'work location', 'out', '18027'],
 ['http://www.wikidata.org/entity/P50', 'author', 'in', '17875'],
 ['http://www.wikidata.org/entity/P166', 'award received', 'out', '15763'],
 ['http://www.wikidata.org/entity/P373', 'Commons category', 'out', '14022'],
 ['http://www.wikidata.org/entity/P108', 'employer', 'out', '9973'],
 ['http://www.wikidata.org/entity/P1343',
  'described by source',
  'out',
  '7873'],
 ['http://www.wikidata.org/entity/P463', 'member of', 'out', '7482'],
 ['http://www.wikidata.org/entity/P40', 'child', 'out', '7019'],
 ['http://www.wikidata.org/entity/P22', 'father', 'in', '6755'],
 ['http://www.wikidata.org/entity/P40', 'child', 'in', '6326'],
 ['http://www.wikidata.org/entity/P119', 'place of burial', 'out', '6306']]
``

## Judge
In this query, we exclude the properties with less of 5000 in number.
In total, There is 68033 instances.

``
[['http://www.wikidata.org/entity/P106', 'occupation', 'out', '72137'],
 ['http://www.wikidata.org/entity/P31', 'instance of', 'out', '36346'],
 ['http://www.wikidata.org/entity/P21', 'sex or gender', 'out', '35775'],
 ['http://www.wikidata.org/entity/P569', 'date of birth', 'out', '34256'],
 ['http://www.wikidata.org/entity/P735', 'given name', 'out', '32566'],
 ['http://www.wikidata.org/entity/P27',
  'country of citizenship',
  'out',
  '30313'],
 ['http://www.wikidata.org/entity/P39', 'position held', 'out', '28204'],
 ['http://www.wikidata.org/entity/P69', 'educated at', 'out', '26290'],
 ['http://www.wikidata.org/entity/P570', 'date of death', 'out', '23809'],
 ['http://www.wikidata.org/entity/P19', 'place of birth', 'out', '21896'],
 ['http://www.wikidata.org/entity/P734', 'family name', 'out', '16528'],
 ['http://www.wikidata.org/entity/P1412',
  'languages spoken, written or signed',
  'out',
  '14787'],
 ['http://www.wikidata.org/entity/P20', 'place of death', 'out', '11530'],
 ['http://www.wikidata.org/entity/P18', 'image', 'out', '9935'],
 ['http://www.wikidata.org/entity/P102',
  'member of political party',
  'out',
  '9263'],
 ['http://www.wikidata.org/entity/P1594', 'judge', 'in', '9088'],
 ['http://www.wikidata.org/entity/P213', 'ISNI', 'out', '8826'],
 ['http://www.wikidata.org/entity/P166', 'award received', 'out', '7878'],
 ['http://www.wikidata.org/entity/P937', 'work location', 'out', '6682'],
 ['http://www.wikidata.org/entity/P1559',
  'name in native language',
  'out',
  '5684'],
 ['http://www.wikidata.org/entity/P40', 'child', 'out', '5660'],
 ['http://www.wikidata.org/entity/P22', 'father', 'in', '5648'],
 ['http://www.wikidata.org/entity/P108', 'employer', 'out', '5084']]
``

## Attorney

In this query, we exclude the properties with less of 100 in number.
In total, There is 100 instances.

``
[['http://www.wikidata.org/entity/P106', 'occupation', 'out', '283'],
 ['http://www.wikidata.org/entity/P31', 'instance of', 'out', '129'],
 ['http://www.wikidata.org/entity/P69', 'educated at', 'out', '125'],
 ['http://www.wikidata.org/entity/P735', 'given name', 'out', '124'],
 ['http://www.wikidata.org/entity/P21', 'sex or gender', 'out', '124'],
 ['http://www.wikidata.org/entity/P27',
  'country of citizenship',
  'out',
  '114'],
 ['http://www.wikidata.org/entity/P569', 'date of birth', 'out', '111'],
 ['http://www.wikidata.org/entity/P19', 'place of birth', 'out', '93'],
 ['http://www.wikidata.org/entity/P570', 'date of death', 'out', '65'],
 ['http://www.wikidata.org/entity/P734', 'family name', 'out', '61'],
 ['http://www.wikidata.org/entity/P20', 'place of death', 'out', '50'],
 ['http://www.wikidata.org/entity/P1412',
  'languages spoken, written or signed',
  'out',
  '46'],
 ['http://www.wikidata.org/entity/P18', 'image', 'out', '45'],
 ['http://www.wikidata.org/entity/P39', 'position held', 'out', '41'],
 ['http://www.wikidata.org/entity/P102',
  'member of political party',
  'out',
  '33'],
 ['http://www.wikidata.org/entity/P213', 'ISNI', 'out', '31'],
 ['http://www.wikidata.org/entity/P373', 'Commons category', 'out', '28'],
 ['http://www.wikidata.org/entity/P108', 'employer', 'out', '22'],
 ['http://www.wikidata.org/entity/P166', 'award received', 'out', '18'],
 ['http://www.wikidata.org/entity/P1343', 'described by source', 'out', '17'],
 ['http://www.wikidata.org/entity/P40', 'child', 'out', '15'],
 ['http://www.wikidata.org/entity/P551', 'residence', 'out', '14'],
 ['http://www.wikidata.org/entity/P1477', 'birth name', 'out', '12'],
 ['http://www.wikidata.org/entity/P463', 'member of', 'out', '12'],
 ['http://www.wikidata.org/entity/P22', 'father', 'in', '11'],
 ['http://www.wikidata.org/entity/P119', 'place of burial', 'out', '11'],
 ['http://www.wikidata.org/entity/P509', 'cause of death', 'out', '11'],
 ['http://www.wikidata.org/entity/P138', 'named after', 'in', '10'],
 ['http://www.wikidata.org/entity/P26', 'spouse', 'out', '10'],
 ['http://www.wikidata.org/entity/P1559',
  'name in native language',
  'out',
  '10'],
 ['http://www.wikidata.org/entity/P921', 'main subject', 'in', '9'],
 ['http://www.wikidata.org/entity/P140', 'religion', 'out', '9'],
 ['http://www.wikidata.org/entity/P1196', 'manner of death', 'out', '9'],
 ['http://www.wikidata.org/entity/P26', 'spouse', 'in', '8'],
 ['http://www.wikidata.org/entity/P856', 'official website', 'out', '8'],
 ['http://www.wikidata.org/entity/P5008',
  'on focus list of Wikimedia project',
  'out',
  '7'],
 ['http://www.wikidata.org/entity/P50', 'author', 'in', '7'],
 ['http://www.wikidata.org/entity/P1442', 'image of grave', 'out', '7'],
 ['http://www.wikidata.org/entity/P1441', 'present in work', 'out', '6'],
 ['http://www.wikidata.org/entity/P112', 'founded by', 'in', '6'],
 ['http://www.wikidata.org/entity/P710', 'participant', 'in', '6'],
 ['http://www.wikidata.org/entity/P937', 'work location', 'out', '6'],
 ['http://www.wikidata.org/entity/P101', 'field of work', 'out', '6'],
 ['http://www.wikidata.org/entity/P1971', 'number of children', 'out', '6'],
 ['http://www.wikidata.org/entity/P25', 'mother', 'in', '5'],
 ['http://www.wikidata.org/entity/P512', 'academic degree', 'out', '5'],
 ['http://www.wikidata.org/entity/P161', 'cast member', 'in', '4'],
 ['http://www.wikidata.org/entity/P674', 'characters', 'in', '4'],
 ['http://www.wikidata.org/entity/P170', 'creator', 'out', '4'],
 ['http://www.wikidata.org/entity/P175', 'performer', 'out', '4'],
 ['http://www.wikidata.org/entity/P1344', 'participant in', 'out', '4'],
 ['http://www.wikidata.org/entity/P40', 'child', 'in', '4'],
 ['http://www.wikidata.org/entity/P172', 'ethnic group', 'out', '4'],
 ['http://www.wikidata.org/entity/P742', 'pseudonym', 'out', '4'],
 ['http://www.wikidata.org/entity/P1950',
  'second family name in Spanish name',
  'out',
  '4'],
 ['http://www.wikidata.org/entity/P973', 'described at URL', 'out', '3'],
 ['http://www.wikidata.org/entity/P641', 'sport', 'out', '3'],
 ['http://www.wikidata.org/entity/P1225',
  'U.S. National Archives Identifier',
  'out',
  '3'],
 ['http://www.wikidata.org/entity/P8687',
  'social media followers',
  'out',
  '3'],
 ['http://www.wikidata.org/entity/P2002', 'Twitter username', 'out', '3'],
 ['http://www.wikidata.org/entity/P22', 'father', 'out', '3'],
 ['http://www.wikidata.org/entity/P6886', 'writing language', 'out', '2'],
 ['http://www.wikidata.org/entity/P1472', 'Commons Creator page', 'out', '2'],
 ['http://www.wikidata.org/entity/P8017', 'generational suffix', 'out', '2'],
 ['http://www.wikidata.org/entity/P1321',
  'place of origin (Switzerland)',
  'out',
  '2'],
 ['http://www.wikidata.org/entity/P136', 'genre', 'out', '2'],
 ['http://www.wikidata.org/entity/P1889', 'different from', 'out', '2'],
 ['http://www.wikidata.org/entity/P109', 'signature', 'out', '2'],
 ['http://www.wikidata.org/entity/P127', 'owned by', 'in', '2'],
 ['http://www.wikidata.org/entity/P97', 'noble title', 'out', '2'],
 ['http://www.wikidata.org/entity/P103', 'native language', 'out', '2'],
 ['http://www.wikidata.org/entity/P485', 'archives at', 'out', '2'],
 ['http://www.wikidata.org/entity/P2031', 'work period (start)', 'out', '2'],
 ['http://www.wikidata.org/entity/P1830', 'owner of', 'out', '2'],
 ['http://www.wikidata.org/entity/P1532', 'country for sport', 'out', '1'],
 ['http://www.wikidata.org/entity/P805',
  'statement is subject of',
  'out',
  '1'],
 ['http://www.wikidata.org/entity/P112', 'founded by', 'out', '1'],
 ['http://www.wikidata.org/entity/P451', 'unmarried partner', 'out', '1'],
 ['http://www.wikidata.org/entity/P361', 'part of', 'out', '1'],
 ['http://www.wikidata.org/entity/P451', 'unmarried partner', 'in', '1'],
 ['http://www.wikidata.org/entity/P57', 'director', 'in', '1'],
 ['http://www.wikidata.org/entity/P25', 'mother', 'out', '1'],
 ['http://www.wikidata.org/entity/P7763',
  'copyright status as a creator',
  'out',
  '1'],
 ['http://www.wikidata.org/entity/P607', 'conflict', 'out', '1'],
 ['http://www.wikidata.org/entity/P241', 'military branch', 'out', '1'],
 ['http://www.wikidata.org/entity/P3373', 'sibling', 'out', '1'],
 ['http://www.wikidata.org/entity/P1889', 'different from', 'in', '1'],
 ['http://www.wikidata.org/entity/P527', 'has part', 'in', '1'],
 ['http://www.wikidata.org/entity/P162', 'producer', 'in', '1'],
 ['http://www.wikidata.org/entity/P3373', 'sibling', 'in', '1'],
 ['http://www.wikidata.org/entity/P344', 'director of photography', 'in', '1'],
 ['http://www.wikidata.org/entity/P157', 'killed by', 'in', '1'],
 ['http://www.wikidata.org/entity/P5821', 'ArhivX LOD', 'out', '1'],
 ['http://www.wikidata.org/entity/P767',
  'contributor to the creative work or subject',
  'in',
  '1'],
 ['http://www.wikidata.org/entity/P54', 'member of sports team', 'out', '1'],
 ['http://www.wikidata.org/entity/P3342', 'significant person', 'out', '1'],
 ['http://www.wikidata.org/entity/P511', 'honorific prefix', 'out', '1'],
 ['http://www.wikidata.org/entity/P2003', 'Instagram username', 'out', '1'],
 ['http://www.wikidata.org/entity/P91', 'sexual orientation', 'out', '1'],
 ['http://www.wikidata.org/entity/P1080',
  'from narrative universe',
  'out',
  '1'],
 ['http://www.wikidata.org/entity/P4966',
  'Hungarian National Assembly identifier',
  'out',
  '1'],
 ['http://www.wikidata.org/entity/P2632', 'place of detention', 'out', '1'],
 ['http://www.wikidata.org/entity/P1268', 'represents', 'out', '1'],
 ['http://www.wikidata.org/entity/P1366', 'replaced by', 'out', '1'],
 ['http://www.wikidata.org/entity/P6241', 'collection creator', 'in', '1'],
 ['http://www.wikidata.org/entity/P5052',
  'supervisory board member',
  'in',
  '1'],
 ['http://www.wikidata.org/entity/P1365', 'replaces', 'in', '1'],
 ['http://www.wikidata.org/entity/P1037', 'director / manager', 'in', '1'],
 ['http://www.wikidata.org/entity/P488', 'chairperson', 'in', '1'],
 ['http://www.wikidata.org/entity/P98', 'editor', 'in', '1'],
 ['http://www.wikidata.org/entity/P767',
  'contributor to the creative work or subject',
  'out',
  '1'],
 ['http://www.wikidata.org/entity/P800', 'notable work', 'out', '1']]
``

## Advocate

In this query, we exclude the properties with less of 100 in number.
In total, There is 702 instances.

``
[['http://www.wikidata.org/entity/P106', 'occupation', 'out', '2177'],
 ['http://www.wikidata.org/entity/P31', 'instance of', 'out', '883'],
 ['http://www.wikidata.org/entity/P21', 'sex or gender', 'out', '876'],
 ['http://www.wikidata.org/entity/P27',
  'country of citizenship',
  'out',
  '813'],
 ['http://www.wikidata.org/entity/P569', 'date of birth', 'out', '748'],
 ['http://www.wikidata.org/entity/P69', 'educated at', 'out', '607'],
 ['http://www.wikidata.org/entity/P39', 'position held', 'out', '527'],
 ['http://www.wikidata.org/entity/P19', 'place of birth', 'out', '515'],
 ['http://www.wikidata.org/entity/P735', 'given name', 'out', '506'],
 ['http://www.wikidata.org/entity/P1412',
  'languages spoken, written or signed',
  'out',
  '434'],
 ['http://www.wikidata.org/entity/P18', 'image', 'out', '383'],
 ['http://www.wikidata.org/entity/P570', 'date of death', 'out', '323'],
 ['http://www.wikidata.org/entity/P102',
  'member of political party',
  'out',
  '271'],
 ['http://www.wikidata.org/entity/P734', 'family name', 'out', '249'],
 ['http://www.wikidata.org/entity/P213', 'ISNI', 'out', '183'],
 ['http://www.wikidata.org/entity/P373', 'Commons category', 'out', '165'],
 ['http://www.wikidata.org/entity/P937', 'work location', 'out', '159'],
 ['http://www.wikidata.org/entity/P20', 'place of death', 'out', '155'],
 ['http://www.wikidata.org/entity/P1592', 'prosecutor', 'in', '137'],
 ['http://www.wikidata.org/entity/P40', 'child', 'out', '137'],
 ['http://www.wikidata.org/entity/P1559',
  'name in native language',
  'out',
  '131'],
 ['http://www.wikidata.org/entity/P22', 'father', 'in', '123'],
 ['http://www.wikidata.org/entity/P166', 'award received', 'out', '109'],
 ['http://www.wikidata.org/entity/P512', 'academic degree', 'out', '107'],
 ['http://www.wikidata.org/entity/P551', 'residence', 'out', '106']]
``

## Magistrate

In this query, we exclude the properties with less of 50 in number.
In total, There is 77 instances.

``
[['http://www.wikidata.org/entity/P106', 'occupation', 'out', '171'],
 ['http://www.wikidata.org/entity/P39', 'position held', 'out', '141'],
 ['http://www.wikidata.org/entity/P735', 'given name', 'out', '125'],
 ['http://www.wikidata.org/entity/P31', 'instance of', 'out', '120'],
 ['http://www.wikidata.org/entity/P21', 'sex or gender', 'out', '117'],
 ['http://www.wikidata.org/entity/P166', 'award received', 'out', '102'],
 ['http://www.wikidata.org/entity/P569', 'date of birth', 'out', '98'],
 ['http://www.wikidata.org/entity/P27', 'country of citizenship', 'out', '87'],
 ['http://www.wikidata.org/entity/P1412',
  'languages spoken, written or signed',
  'out',
  '84'],
 ['http://www.wikidata.org/entity/P19', 'place of birth', 'out', '75'],
 ['http://www.wikidata.org/entity/P734', 'family name', 'out', '72'],
 ['http://www.wikidata.org/entity/P570', 'date of death', 'out', '53'],
 ['http://www.wikidata.org/entity/P69', 'educated at', 'out', '43'],
 ['http://www.wikidata.org/entity/P20', 'place of death', 'out', '36'],
 ['http://www.wikidata.org/entity/P213', 'ISNI', 'out', '25'],
 ['http://www.wikidata.org/entity/P1477', 'birth name', 'out', '23'],
 ['http://www.wikidata.org/entity/P463', 'member of', 'out', '21'],
 ['http://www.wikidata.org/entity/P108', 'employer', 'out', '18'],
 ['http://www.wikidata.org/entity/P18', 'image', 'out', '14'],
 ['http://www.wikidata.org/entity/P1559',
  'name in native language',
  'out',
  '11']]
``

## Advocate in France

In this query, we exclude the properties with less of 50 in number.
In total, There is 29 instances.

``
[['http://www.wikidata.org/entity/P106', 'occupation', 'out', '55'],
 ['http://www.wikidata.org/entity/P39', 'position held', 'out', '55'],
 ['http://www.wikidata.org/entity/P69', 'educated at', 'out', '47'],
 ['http://www.wikidata.org/entity/P1412',
  'languages spoken, written or signed',
  'out',
  '33'],
 ['http://www.wikidata.org/entity/P735', 'given name', 'out', '32'],
 ['http://www.wikidata.org/entity/P166', 'award received', 'out', '30'],
 ['http://www.wikidata.org/entity/P569', 'date of birth', 'out', '29'],
 ['http://www.wikidata.org/entity/P31', 'instance of', 'out', '29'],
 ['http://www.wikidata.org/entity/P21', 'sex or gender', 'out', '29'],
 ['http://www.wikidata.org/entity/P27', 'country of citizenship', 'out', '28'],
 ['http://www.wikidata.org/entity/P734', 'family name', 'out', '24'],
 ['http://www.wikidata.org/entity/P19', 'place of birth', 'out', '24'],
 ['http://www.wikidata.org/entity/P101', 'field of work', 'out', '19'],
 ['http://www.wikidata.org/entity/P213', 'ISNI', 'out', '14'],
 ['http://www.wikidata.org/entity/P570', 'date of death', 'out', '11'],
 ['http://www.wikidata.org/entity/P463', 'member of', 'out', '10'],
 ['http://www.wikidata.org/entity/P108', 'employer', 'out', '10'],
 ['http://www.wikidata.org/entity/P1559',
  'name in native language',
  'out',
  '9'],
 ['http://www.wikidata.org/entity/P18', 'image', 'out', '9'],
 ['http://www.wikidata.org/entity/P20', 'place of death', 'out', '8'],
 ['http://www.wikidata.org/entity/P937', 'work location', 'out', '7'],
 ['http://www.wikidata.org/entity/P373', 'Commons category', 'out', '7'],
 ['http://www.wikidata.org/entity/P103', 'native language', 'out', '7'],
 ['http://www.wikidata.org/entity/P102',
  'member of political party',
  'out',
  '6'],
 ['http://www.wikidata.org/entity/P50', 'author', 'in', '5'],
 ['http://www.wikidata.org/entity/P2031', 'work period (start)', 'out', '5'],
 ['http://www.wikidata.org/entity/P1477', 'birth name', 'out', '5'],
 ['http://www.wikidata.org/entity/P3602', 'candidacy in election', 'out', '3'],
 ['http://www.wikidata.org/entity/P973', 'described at URL', 'out', '3'],
 ['http://www.wikidata.org/entity/P1038', 'relative', 'in', '3'],
 ['http://www.wikidata.org/entity/P488', 'chairperson', 'in', '3'],
 ['http://www.wikidata.org/entity/P40', 'child', 'in', '3'],
 ['http://www.wikidata.org/entity/P241', 'military branch', 'out', '3'],
 ['http://www.wikidata.org/entity/P485', 'archives at', 'out', '3'],
 ['http://www.wikidata.org/entity/P512', 'academic degree', 'out', '3'],
 ['http://www.wikidata.org/entity/P22', 'father', 'out', '3'],
 ['http://www.wikidata.org/entity/P8687',
  'social media followers',
  'out',
  '3'],
 ['http://www.wikidata.org/entity/P2002', 'Twitter username', 'out', '3'],
 ['http://www.wikidata.org/entity/P812', 'academic major', 'out', '2'],
 ['http://www.wikidata.org/entity/P6886', 'writing language', 'out', '2'],
 ['http://www.wikidata.org/entity/P1472', 'Commons Creator page', 'out', '2'],
 ['http://www.wikidata.org/entity/P40', 'child', 'out', '2'],
 ['http://www.wikidata.org/entity/P1038', 'relative', 'out', '2'],
 ['http://www.wikidata.org/entity/P527', 'has part', 'in', '2'],
 ['http://www.wikidata.org/entity/P185', 'doctoral student', 'in', '2'],
 ['http://www.wikidata.org/entity/P22', 'father', 'in', '2'],
 ['http://www.wikidata.org/entity/P1196', 'manner of death', 'out', '2'],
 ['http://www.wikidata.org/entity/P509', 'cause of death', 'out', '2'],
 ['http://www.wikidata.org/entity/P800', 'notable work', 'out', '2'],
 ['http://www.wikidata.org/entity/P742', 'pseudonym', 'out', '2'],
 ['http://www.wikidata.org/entity/P184', 'doctoral advisor', 'out', '2'],
 ['http://www.wikidata.org/entity/P2032', 'work period (end)', 'out', '1'],
 ['http://www.wikidata.org/entity/P1889', 'different from', 'out', '1'],
 ['http://www.wikidata.org/entity/P553', 'website account on', 'out', '1'],
 ['http://www.wikidata.org/entity/P140', 'religion', 'out', '1'],
 ['http://www.wikidata.org/entity/P1889', 'different from', 'in', '1'],
 ['http://www.wikidata.org/entity/P1855',
  'Wikidata property example',
  'in',
  '1'],
 ['http://www.wikidata.org/entity/P1037', 'director / manager', 'in', '1'],
 ['http://www.wikidata.org/entity/P921', 'main subject', 'in', '1'],
 ['http://www.wikidata.org/entity/P161', 'cast member', 'in', '1'],
 ['http://www.wikidata.org/entity/P4033', 'Mastodon address', 'out', '1'],
 ['http://www.wikidata.org/entity/P1581', 'official blog', 'out', '1'],
 ['http://www.wikidata.org/entity/P3373', 'sibling', 'in', '1'],
 ['http://www.wikidata.org/entity/P2679', 'author of foreword', 'in', '1'],
 ['http://www.wikidata.org/entity/P1591', 'defendant', 'in', '1'],
 ['http://www.wikidata.org/entity/P664', 'organizer', 'in', '1'],
 ['http://www.wikidata.org/entity/P26', 'spouse', 'in', '1'],
 ['http://www.wikidata.org/entity/P7763',
  'copyright status as a creator',
  'out',
  '1'],
 ['http://www.wikidata.org/entity/P3373', 'sibling', 'out', '1'],
 ['http://www.wikidata.org/entity/P410', 'military rank', 'out', '1'],
 ['http://www.wikidata.org/entity/P26', 'spouse', 'out', '1'],
 ['http://www.wikidata.org/entity/P793', 'significant event', 'out', '1'],
 ['http://www.wikidata.org/entity/P641', 'sport', 'out', '1'],
 ['http://www.wikidata.org/entity/P7060',
  'NosSénateurs.fr identifier',
  'out',
  '1']]
``